<a href="https://colab.research.google.com/github/tg-bomze/collection-of-notebooks/blob/master/QuickFaceSwap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<b><font color="black" size="+4">QuickFaceSwap v.1.1</font></b>

<b><font color="black" size="+2">Based on:</font></b>

**GitHub repository**: [SimSwap](https://github.com/neuralchen/SimSwap), [GFPGAN](https://github.com/TencentARC/GFPGAN)

Article: [SimSwap: An Efficient Framework For High Fidelity Face Swapping](https://arxiv.org/pdf/2106.06340v1.pdf), [Towards Real-World Blind Face Restoration with Generative Facial Prior
](https://arxiv.org/pdf/2101.04061.pdf)

**Creators of SimSwap:** *Renwang Chen, Xuanhong Chen, Bingbing Ni, Yanhao Ge*

**Creators of GFPGAN:** *Xintao Wang and Yu Li and Honglun Zhang and Ying Shan*

<b><font color="black" size="+2">Colab created by:</font></b>

GitHub: [@tg-bomze](https://github.com/tg-bomze),
Telegram: [@bomze](https://t.me/bomze),
Twitter: [@tg_bomze](https://twitter.com/tg_bomze).

---


```
(ENG) To get started, click on the button (where the red arrow indicates). After clicking, wait until the execution is complete.
```
```
(RUS) Чтобы начать, поочередно нажимайте на кнопки (куда указывают красные стрелки), дожидаясь завершения выполнения каждого блока.
```


In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Install all necessary libraries</font></b>
#@markdown <b><font color="black" size="+1"> (RUS) Установить все необходимые библиотеки</font></b>

#@markdown **Follow this link (перейдите по ссылке):**

#@markdown **https://drive.google.com/drive/folders/1jV6_0FIMPC53FZ2HzZNJZGMe55bbu17R**

#@markdown **1) right click on 'checkpoints' (правой кнопкой по checkpoints)**

#@markdown **2) select 'Add shortcut to Drive (выберите Сохранить ярлык на Диск)**

#@markdown ![](https://github.com/tg-bomze/collection-of-notebooks/raw/master/dfs.png)

#@markdown **3) run this block and follow the further instructions (после этого запустите блок и следуйте инструкции)**

#@markdown *Attention! If the weights have already been saved, then run this block and just mount Google Drive.*

#@markdown *Внимание! Если веса уже сохранены, то можете сразу запустить блок и смонтировать Google Drive.*

from google.colab import drive
from google.colab import files
from IPython.display import clear_output
import os
drive.mount('/content/drive')
!pip install ffmpeg-python
!pip install facexlib

%cd /content
import torch
!BASICSR_EXT=True pip install basicsr
!git clone https://github.com/TencentARC/GFPGAN.git
%cd /content/GFPGAN
!mkdir MODNet
!pip install -r requirements.txt
!wget https://github.com/TencentARC/GFPGAN/releases/download/v0.1.0/GFPGANv1.pth -P experiments/pretrained_models
!wget https://www.dropbox.com/s/0zwrook2mxxb4vc/modnet_photographic_portrait_matting.ckpt
%cd /content/GFPGAN/MODNet
!wget https://www.dropbox.com/s/4nwzaaq5x1tm2ze/MODNet.zip
!unzip /content/GFPGAN/MODNet/MODNet.zip
!rm -rf /content/GFPGAN/MODNet/MODNet.zip

%cd /content
!git clone https://github.com/neuralchen/SimSwap
%cd /content/SimSwap
!mkdir -p /usr/local/lib/python3.7/dist-packages/facexlib/weights
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bzip2 -dk shape_predictor_68_face_landmarks.dat.bz2
!rm -rf shape_predictor_68_face_landmarks.dat.bz2
!mkdir arcface_model checkpoints
%cd /content/SimSwap/checkpoints
!cp /content/drive/MyDrive/checkpoint/checkpoints.zip ./
!unzip /content/SimSwap/checkpoints/checkpoints.zip
%cd /content/SimSwap/arcface_model
!cp /content/drive/MyDrive/checkpoint/arcface_checkpoint.tar ./
%cd /content/SimSwap

path_to_frames = '/content/frames'
path_to_result = '/content/result'
!rm -rf $path_to_frames $path_to_result /content/sample_data
!mkdir $path_to_frames $path_to_result

clear_output()

In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Upload video</font></b>
#@markdown <b><font color="black" size="+1"> (RUS) Загрузить видео</font></b>

import os
import cv2
import ffmpeg

uploaded = files.upload()
for vp in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=vp, length=len(uploaded[vp])))
video_path = f'/content/SimSwap/{vp}'

!rm -rf /content/frames/*
!ffmpeg -y -i $video_path /content/audio.mp3
!ffmpeg -i $video_path /content/frames/%7d.png

fps_of_video = int(cv2.VideoCapture(video_path).get(cv2.CAP_PROP_FPS))
fps_of_video +=1

if os.listdir('/content/frames'):
  clear_output()
  print('Done!')
else: print('Error!')

In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Upload photo with face</font></b>
#@markdown <b><font color="black" size="+1"> (RUS) Загрузить фото с лицом</font></b>
uploaded = files.upload()
for pp in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=pp, length=len(uploaded[pp])))
clear_output()
print('Done!')

In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Prepare script</font></b>
#@markdown <b><font color="black" size="+1"> (RUS) Подготовить скрипт</font></b>
%%writefile /content/SimSwap/test_video.py
import torch
import fractions
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
import PIL
from PIL import Image
from tqdm import tqdm
import warnings
import numpy as np
import imageio
import glob
import os
import sys
import cv2
import scipy
import scipy.ndimage
import dlib

size = 224
path_to_frames = '/content/frames'
path_to_result = '/content/result'


sys.path.append("/content/GFPGAN")
os.chdir('/content/GFPGAN')
from MODNet.models.modnet import MODNet
from archs.gfpganv1_arch import GFPGANv1
from basicsr.utils import img2tensor, imwrite, tensor2img
from torchvision.transforms.functional import normalize
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

weight = "/content/GFPGAN/modnet_photographic_portrait_matting.ckpt"
im_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
modnet = MODNet(backbone_pretrained=False)
modnet = nn.DataParallel(modnet).cuda()
modnet.load_state_dict(torch.load(weight))
modnet.eval()


print('Loading weights ...')
gfpgan = GFPGANv1(
        out_size=512,
        num_style_feat=512,
        channel_multiplier=1,
        decoder_load_path=None,
        fix_decoder=True,
        num_mlp=8,
        input_is_latent=True,
        different_w=True,
        narrow=1,
        sft_half=True)
gfpgan.to(device)
checkpoint = torch.load('/content/GFPGAN/experiments/pretrained_models/GFPGANv1.pth', map_location=lambda storage, loc: storage)
gfpgan.load_state_dict(checkpoint['params_ema'])
gfpgan.eval()


sys.path.append("/content/SimSwap")
os.chdir('/content/SimSwap')
from models.models import create_model
from options.test_options import TestOptions
from facexlib.utils.face_restoration_helper import FaceRestoreHelper

transformer = transforms.Compose([
        transforms.ToTensor(),
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

transformer_Arcface = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

detransformer = transforms.Compose([
        transforms.Normalize([0, 0, 0], [1/0.229, 1/0.224, 1/0.225]),
        transforms.Normalize([-0.485, -0.456, -0.406], [1, 1, 1])
    ])

opt = TestOptions().parse()
path_to_face = opt.pic_a_path
start_epoch, epoch_iter = 1, 0
torch.nn.Module.dump_patches = True
model = create_model(opt)
model.eval()

def post_process(color, orig):
        orig_yuv = cv2.cvtColor(color, cv2.COLOR_BGR2YUV)
        hires = np.copy(orig_yuv)
        hires[:, :, 1:3] = orig[:, :, 1:3]
        final = cv2.cvtColor(hires, cv2.COLOR_YUV2BGR)
        return final

def swapping(face_helper, img_path, latend_id, img_id, save_root):
    img_name = os.path.basename(img_path)
    basename, _ = os.path.splitext(img_name)
    input_img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    face_helper.clean_all()

    face_helper.read_image(input_img)
    face_helper.get_face_landmarks_5(only_center_face=True, pad_blur=False)
    save_crop_path = os.path.join(save_root, 'cropped_faces', img_name)
    face_helper.align_warp_face(save_crop_path)

    for idx, cropped_face in enumerate(face_helper.cropped_faces):
      small_cropped_face = cv2.resize(cropped_face, (size,size))
      pil_small_cropped_face = Image.fromarray(small_cropped_face).convert('RGB')
      img_b = transformer(pil_small_cropped_face)
      img_att = img_b.view(-1, img_b.shape[0], img_b.shape[1], img_b.shape[2])
      img_att = img_att.cuda()

      img_fake = model(img_id, img_att, latend_id, latend_id, True)
      for i in range(img_id.shape[0]):
          if i == 0:
              row1 = img_id[i]
              row2 = img_att[i]
              row3 = img_fake[i]
          else:
              row1 = torch.cat([row1, img_id[i]], dim=2)
              row2 = torch.cat([row2, img_att[i]], dim=2)
              row3 = torch.cat([row3, img_fake[i]], dim=2)

      full = row3.detach()
      full = full.permute(1, 2, 0)
      output = full.to('cpu')
      output = np.array(output)
      output = output[..., ::-1]
      output = cv2.resize(output, (512,512))
      yuv_swapped_face = cv2.cvtColor(cropped_face, cv2.COLOR_BGR2YUV)

      cropped_face_t = img2tensor(output, bgr2rgb=False, float32=True)
      normalize(cropped_face_t, (0.5, 0.5, 0.5), (0.5, 0.5, 0.5), inplace=True)
      cropped_face_t = cropped_face_t.unsqueeze(0).to('cuda')

      try:
        with torch.no_grad():
          output = gfpgan(cropped_face_t, return_rgb=False)[0]
          restored_face = tensor2img(output.squeeze(0), rgb2bgr=True, min_max=(-1, 1))
      except RuntimeError as error: restored_face = cropped_face

      final_face = post_process(restored_face, yuv_swapped_face)

      im = im_transform(final_face)
      im = im[None, :, :, :]
      im_b, im_c, im_h, im_w = im.shape
      im_rw = im_w - im_w % 32
      im_rh = im_h - im_h % 32
      im = F.interpolate(im, size=(im_rh, im_rw), mode='area')
      _, _, matte = modnet(im.cuda(), True)
      matte = F.interpolate(matte, size=(im_h, im_w), mode='area')
      matte = matte[0][0].data.cpu().numpy()
      matte = np.repeat(np.asarray(matte)[:, :, None], 3, axis=2)
      out = final_face.copy()
      out[matte<0.5] = cropped_face[matte<0.5]


      face_helper.add_restored_face(out)

    face_helper.get_inverse_affine(None)
    save_restore_path = os.path.join(save_root, 'restored_imgs', img_name)
    face_helper.paste_faces_to_input_image(save_restore_path)

source_helper = FaceRestoreHelper(face_size=512, upscale_factor=1, crop_ratio=(1, 1), det_model='retinaface_resnet50', save_ext='png')

source_helper.read_image(path_to_face)
source_helper.get_face_landmarks_5(only_center_face=True, pad_blur=False)
save_crop_path = os.path.join(path_to_result, 'cropped_faces', os.path.basename(path_to_face))
source_helper.align_warp_face(save_crop_path)

path_to_croped_src = '/content/result/cropped_faces/'
path_to_cropped_src = path_to_croped_src+os.listdir(path_to_croped_src)[0]
cropped_src = Image.open(path_to_cropped_src).convert('RGB')
small_cropped_src = cropped_src.resize((size,size), Image.ANTIALIAS)

img_a = transformer_Arcface(small_cropped_src)
img_id = img_a.view(-1, img_a.shape[0], img_a.shape[1], img_a.shape[2])
img_id = img_id.cuda()
img_id_downsample = F.interpolate(img_id, scale_factor=0.5)
latend_id = model.netArc(img_id_downsample)
latend_id = latend_id.detach().to('cpu')
latend_id = latend_id/np.linalg.norm(latend_id,axis=1,keepdims=True)
latend_id = latend_id.to('cuda')

face_helper = FaceRestoreHelper(face_size=512, upscale_factor=1, crop_ratio=(1, 1), det_model='retinaface_resnet50', save_ext='png')
img_list = sorted(glob.glob(os.path.join(path_to_frames, '*')))
for img_path in tqdm(img_list):
  swapping(face_helper, img_path, latend_id, img_id, path_to_result)

In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Process</font></b>
#@markdown <b><font color="black" size="+1"> (RUS) Стартуем</font></b>
path_to_face_photo = f'/content/SimSwap/{pp}'
!rm -rf /content/result/cropped_faces/* /content/result/restored_imgs/* /content/result/cropped_faces/.ipynb_checkpoints
!python test_video.py --Arc_path /content/drive/MyDrive/checkpoint/arcface_checkpoint.tar --pic_a_path $path_to_face_photo
if os.listdir('/content/result/restored_imgs'):
  clear_output()
  print('Done!')
else: print('Error!')

In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Merge frames into video</font></b>
#@markdown <b><font color="black" size="+1"> (RUS) Объединяем кадры в видео</font></b>
video_type = "good for sharing" #@param ["good for sharing", "good for editing"]
if video_type == "good for sharing":
  !ffmpeg -y -framerate $fps_of_video -pattern_type glob -i '/content/result/restored_imgs/*.png' -i /content/audio.mp3 -c:a copy -shortest -c:v libx264 -r $fps_of_video -pix_fmt yuv420p /content/swapped.mp4
else:
  !ffmpeg -y -r $fps_of_video -i /content/result/restored_imgs/%7d.png -i /content/audio.mp3 /content/swapped.mp4
clear_output()
files.download('/content/swapped.mp4')



---

**TODO:**


`Done` - Decrease the brightness of the face square

`Done` -   Make a round mask

`In process` - Make the right Frames2Video

